In [1]:
import sys
sys.path.insert(1, '../Source_Code')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import importlib
import pandas as pd
import seaborn
from IPython.display import Image
import matplotlib.pyplot as plt

import Transformer as tnsf
import preprocess as ad

import copy
from matplotlib.colors import LogNorm
np.set_printoptions(threshold=sys.maxsize)

importlib.reload(ad)
importlib.reload(tnsf)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cuda"
print(device)

cuda


In [2]:
def test_generate(name, window_size=10):
    hdfs = set()
    # hdfs = []
    with open( name, 'r') as f:
        for ln in f.readlines():
            ln = list(map(lambda n: n, map(int, ln.strip().split())))
#             ln = ln + [0] * (window_size + 1 - len(ln))
            hdfs.add(tuple(ln))
            # hdfs.append(tuple(ln))
    print('Number of sessions({}): {}'.format(name, len(hdfs)))
    return hdfs

In [3]:
importlib.reload(ad)

<module 'preprocess' from '../Source_Code/preprocess.py'>

In [4]:
def create_mat(dataloader, model, layers, heads, file, enc =True, dec = False):
    
    enc_att = np.zeros((449,449))
    enc_tp_att = np.zeros((449,449))
    enc_fn_att = np.zeros((449,449))
    
    if dec:
        dec_att = np.zeros((449,449))
        dec_tp_att = np.zeros((449,449))
        dec_fn_att = np.zeros((449,449))
    
    src_mask = Variable(torch.ones(1, 1, window_size + 1)).to(device)
    bos = torch.ones((1, ),dtype = int).to(device)
    
    print("********** ********** ********** ********** **********")
    print("      ********** FIRST INPUT SEQUENCE ********** ")
    print("********** ********** ********** ********** **********")
    print("\n")
    sequence_cnt = 10
    anomaly_timeline = []
    line_number = 1
    for line in dataloader:
        if line_number%25==0:
            print("Current Sequence Number:",line_number)
        else:
            pass
        line_number += 1
        for i in range(len(line) - window_size):
            seq = line[i:i + window_size]
            
            #for predicting 10
            label = line[i+window_size:(i+window_size)+window_size]
            
            #for predicting only 1
            ###label = line[i+window_size:(i+window_size)+1]
            ###print("Sequence: {}".format(seq))
            ###ad.backtrace(seq, "linux.log", "Spell")
            

            t1 = torch.cat((bos, torch.tensor(seq, dtype = torch.int).to(device))).unsqueeze(0)
            t2 = torch.tensor(label, dtype = torch.int).to(device).unsqueeze(0)

            src = Variable(t1, requires_grad =False)
            tgt = Variable(t2, requires_grad =False)

            print("---------- ---------- ---------- ---------- ----------")
            print("Input Sequence: {}".format(seq))
            print("Label: {}".format(label))
            print("\n")

#             print("~~~~~~~~~~ ATTENTION WEIGHTS ON GREEDY DECODE ~~~~~~~~~~")
#             print("\n")
            ###tgt_pred = tnsf.predict(model, src, src_mask, tgt, max_len = len(tgt)+1, start_symbol = 1, g = 10)
            ###tgt_pred = tnsf.greedy_decode(model, src, src_mask, tgt, len(label)+1, 1, True, g=10, halt = True, layers = layers, heads = heads)
            tgt_pred, candidates = tnsf.greedy_decode(model, src, src_mask, tgt, 2, 1, True, g=10, halt = True, layers = layers, heads = heads)

            # List for every deteected anomaly
            if tgt_pred.cpu().detach().numpy() in candidates.cpu().detach().numpy():
                anomaly_timeline.append(0)
            else:
                anomaly_timeline.append(1)
                
            ###print("---------- ---------- ---------- ---------- ----------")
            ###print("Input Sequence: {}".format(seq))
            ###print("Label: {}".format(label))
            ###print("Pred: {}".format(tgt_pred[0][0]))

            print("\n")
            print("~~~~~~~~~~ SEQUENCE BACKTRACE ~~~~~~~~~~")
            ad.backtrace(seq, "linux.log", "Spell")
            print("\n")
            print("~~~~~~~~~~ LABEL BACKTRACE ~~~~~~~~~~")
            ad.backtrace(label[:1], "linux.log", "Spell") 
            print("\n")
            print("~~~~~~~~~~ PREDICTION BACKTRACE [Top 4] ~~~~~~~~~~")
            ###ad.backtrace(tuple(tgt_pred.cpu().detach().numpy()[0]) , "linux.log", "Spell")
            ad.backtrace(tuple(np.flip(candidates.cpu().detach().numpy()[6:])) , "linux.log", "Spell")
            print("\n")
            
            enc_weights = 0
            
            for layer in range(layers):
                for h in range(heads):
                    ###print("Layer:{} , Head: {}".format(layer,h))
                    enc_weights += model.encoder.layers[layer].self_attn.attn[0, h].data[:len(label), 1:len(seq)+1][0]
                    ###print("~~~~~~~~~~ ENCODER ATTENTION WEIGHTS ~~~~~~~~~~")
                    ###print(enc_weights)
                    ###print("\n")
                    
            if dec:
                dec_weights = 0
                for layer in range(layers):
                    for h in range(heads):
                        ###print("Decoder Layer:{} , Head: {}".format(layer,h))                    
                        dec_weights += model.decoder.layers[layer].self_attn.attn[0, h].data[:len(label), :len(seq)][0]
                        ###if you are predicting 10 use the code below
                        ###dec_weights += model.decoder.layers[layer].self_attn.attn[0, h].data[:len(label), :len(seq)]
                        ###print("~~~~~~~~~~ DECODER ATTENTION WEIGHTS ~~~~~~~~~~")
                        ###print(dec_weights)
                        ###print("\n")
            
            print("########## END OF INFO FOR LOG SEQUENCE {} ##########".format(sequence_cnt))
            print("\n")
            print("\n")
            print("\n")

            sequence_cnt += 1
            
            print("********** ********** ********** ********** **********")
            print("     ********** INPUT SEQUENCE {} - {} **********".format(sequence_cnt-9,sequence_cnt))
            print("********** ********** ********** ********** **********")
            print("\n")
            
            ###print(label[0],tgt_pred.cpu().detach().numpy())
            ###print(label[0] in tgt_pred.cpu().detach().numpy())
            ###if label[0] not in tgt_pred.cpu().detach().numpy():
            ###    print(label[0],tgt_pred)
            ###else:
            ###    pass
            
            for i in range(len(tgt_pred)):
#             uncomment if you are using 10 predictions instead of 1
                for j in range(len(tgt_pred)):
                        enc_att[seq[i],label[j]] += enc_weights[i]  
#                 enc_att[seq[i], label[0]] += enc_weights[i]

            
            if -1 in tgt_pred:
                for i in range(len(tgt_pred)):
                #uncomment if you are using 10 predictions instead of 1
                    for j in range(len(tgt_pred)):
                        enc_tp_att[seq[i],label[j]] += enc_weights[i]
#                     enc_tp_att[seq[i],label[0]] += enc_weights[i]
            else:
                for i in range(len(tgt_pred)):
                #uncomment if you are using 10 predictions instead of 1
                    for j in range(len(tgt_pred)):
                        enc_fn_att[seq[i],label[j]] += enc_weights[i]    
#                     enc_fn_att[seq[i], label[0]] += enc_weights[i]
            
            if dec:
                for i in range(len(tgt_pred)):
                    for j in range(len(tgt_pred)):
                        dec_att[seq[i], label[j]] += dec_weights[i]

                if -1 in tgt_pred:
#                    print("halt")
                    for i in range(len(tgt_pred)):
                        for j in range(len(tgt_pred)):
                            dec_tp_att[seq[i],label[j]] += dec_weights[i]
                else:
                    for i in range(len(tgt_pred)):
                        for j in range(len(tgt_pred)):
                            dec_fn_att[seq[i], label[j]] += dec_weights[i]    
            

#     with open("Attention-Based_Weights/all_" + file + '_attention.npy', 'wb') as f:
#         np.save(f, enc_att)
                        
#     with open("Attention-Based_Weights/tp_" + file + '_attention.npy', 'wb') as f:
#         np.save(f, enc_tp_att)
        
#     with open("Attention-Based_Weights/fn_" + file + '_attention.npy', 'wb') as f:
#         np.save(f, enc_fn_att)        
    
    enc_att_normalized =(enc_att-np.min(enc_tp_att))/(np.max(enc_att)-np.min(enc_att))
    enc_tp_att_normalized =(enc_tp_att-np.min(enc_tp_att))/(np.max(enc_tp_att)-np.min(enc_tp_att))
    enc_fn_att_normalized =(enc_fn_att-np.min(enc_fn_att))/(np.max(enc_fn_att)-np.min(enc_fn_att))
#     print(enc_fn_att_normalized[enc_fn_att_normalized>0])
    if dec:
        dec_att_normalized =(dec_att-np.min(dec_tp_att))/(np.max(dec_att)-np.min(dec_att))
        dec_tp_att_normalized =(dec_tp_att-np.min(dec_tp_att))/(np.max(dec_tp_att)-np.min(dec_tp_att))
        dec_fn_att_normalized =(dec_fn_att-np.min(dec_fn_att))/(np.max(dec_fn_att)-np.min(dec_fn_att))

#     with open("Attention-Based_Weights/all_" + file + '_normalized_attention.npy', 'wb') as f:
#             np.save(f, enc_att_normalized)  
#     with open("Attention-Based_Weights/tp_" + file + '_normalized_attention.npy', 'wb') as f:
#             np.save(f, enc_tp_att_normalized)  
#     with open("Attention-Based_Weights/fn_" + file + '_normalized_attention.npy', 'wb') as f:
#             np.save(f, enc_fn_att_normalized)              

    # Plot Anomaly Detection Timeline
    fig = plt.figure(figsize = (15, 5))
    x = range(11,len(anomaly_timeline)+11)
    plt.xlabel("Log Timeline", fontsize=14)
    plt.ylabel("Anomaly = 1, Normal =0", fontsize=14)
    plt.title("Anomaly Detection Timelline", fontsize=14)
    plt.bar(x, anomaly_timeline, color = "blue")
    plt.xlim([11, len(anomaly_timeline)+11])
    #plt.setp(axs[1], yscale = 'log', xlabel="Log Key (Input Sequences)")
    plt.tight_layout()
    # plt.savefig('GonnaCry_bar_all.png')
    plt.show()        
            
#     return (enc_att_normalized, enc_tp_att_normalized, enc_fn_att_normalized), (dec_att_normalized, dec_tp_att_normalized, dec_fn_att_normalized)
    return (enc_att_normalized, enc_tp_att_normalized, enc_fn_att_normalized)

In [5]:
test_abnormal_loader = test_generate("../CTDD_Dataset/Sample_Cyber_Threat_Log_Keys/jy-ntp-ddos-amplification-atk-tool-t1498")

Number of sessions(../CTDD_Dataset/Sample_Cyber_Threat_Log_Keys/jy-ntp-ddos-amplification-atk-tool-t1498): 1


In [6]:
model = torch.load("../Saved_Models/global_models_practicum.pt")

In [7]:
# Model information
layers = 1
heads = 1
WINDOW_SIZE = 10
window_size = 10
importlib.reload(ad)
importlib.reload(tnsf)

<module 'Transformer' from '../Source_Code/Transformer.py'>

In [8]:
#print backtrack of logs
#tp att are for true positives
#fn att are for false negatives

att, tp_att, fn_att = create_mat(test_abnormal_loader, model, layers, heads, "ntp-ddos", enc=True, dec=False)


********** ********** ********** ********** **********
      ********** FIRST INPUT SEQUENCE ********** 
********** ********** ********** ********** **********


---------- ---------- ---------- ---------- ----------
Input Sequence: (418, 258, 259, 224, 260, 260, 260, 260, 260, 260)
Label: (260, 260, 263, 264, 264, 264, 164, 265, 266, 267)


^^^^^^^^^^ ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Candidate logs:  tensor([ 20, 263, 164,  51,  32, 149, 260, 264,   8,   7], device='cuda:0')


~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~
Probabilities:  tf.Tensor([0.0169 0.0212 0.0230 0.0320 0.0370 0.0389 0.1109 0.1226 0.1936 0.4039], shape=(10,), dtype=float32) 



~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~ ~~~~~~~~~~
Encoder Attention Normalized [0.2622 0.7205 0.1896 0.9971 0.0000 1.0000 0.0000 0.9778 0.0000 0.7124]




~~~~~~~~~~ SEQUENCE BACKTRACE ~~~~~~~~~~


../Source_Code/Transformer.py:495: RuntimeWarning: invalid value encountered in true_divide
  normalized_weights =(att-np.min(att))/(np.max(att)-np.min(att))


FileNotFoundError: [Errno 2] No such file or directory: '../Source_Code/Spell_result/linux.log_templates.csv'